In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install catboost

     |████████████████████████████████| 63.9MB 61kB/s 


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb
import holidays
from datetime import date
import re
from catboost import CatBoostRegressor
%matplotlib inline

In [0]:
train = pd.read_csv('/content/drive/My Drive/AV LTFS Finhack2/train.csv', parse_dates=['application_date'])
test = pd.read_csv('/content/drive/My Drive/AV LTFS Finhack2/test.csv', parse_dates=['application_date'])

In [0]:
train_sum = train.groupby(by=['application_date', 'segment'])['case_count'].sum().reset_index().rename(columns={"case_count": "case_count_sum"})
train = train.merge(train_sum, on=['application_date', 'segment'], how='left')
train.shape

(80402, 7)

In [0]:
holidays = holidays.India(years=[2017, 2018, 2019])
holidays.append('2017-03-28')
holidays.append('2017-09-30')
holidays.append('2017-10-19')
holidays.append('2018-03-02')
holidays.append('2018-03-18')
holidays.append('2018-09-13')
holidays.append('2018-10-19')
holidays.append('2018-11-07')
holidays.append('2019-03-21')
holidays.append('2019-04-06')
holidays.append('2019-09-02')
holidays.append('2019-10-08')
holidays.append('2019-10-27')

In [0]:
train['is_holiday'] = train['application_date'].dt.date.isin(holidays)

In [0]:
def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [0]:
train.drop_duplicates(['application_date', 'segment'], inplace=True)
train.shape

(1650, 8)

In [0]:
add_datepart(train, 'application_date')

In [0]:
train.drop(['branch_id', 'state', 'zone'], axis=1, inplace=True)

In [0]:
train.drop(['case_count'], axis=1, inplace=True)
train.rename(columns={'case_count_sum': 'case_count'}, inplace=True)

In [0]:
train['case_count'] = np.log1p(train['case_count'])

In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(train.drop(['case_count'], axis=1), train['case_count'], 
                                                    test_size=0.2, random_state=423)

In [0]:
categorical_features = train.select_dtypes(include=np.bool)
categorical_features.columns

Index(['is_holiday', 'application_Is_month_end', 'application_Is_month_start',
       'application_Is_quarter_end', 'application_Is_quarter_start',
       'application_Is_year_end', 'application_Is_year_start'],
      dtype='object')

In [0]:
model = CatBoostRegressor(n_estimators = 1000, depth=10,
    loss_function = 'MAPE',
    eval_metric = 'MAPE',
    cat_features = categorical_features.columns )

In [0]:
model.fit(X_train, y_train, use_best_model=True, eval_set=(X_valid, y_valid), early_stopping_rounds=50)

0:	learn: 0.1555634	test: 0.1244017	best: 0.1244017 (0)	total: 47.7ms	remaining: 47.7s
1:	learn: 0.1526784	test: 0.1216840	best: 0.1216840 (1)	total: 82ms	remaining: 40.9s
2:	learn: 0.1498109	test: 0.1190483	best: 0.1190483 (2)	total: 117ms	remaining: 39s
3:	learn: 0.1469957	test: 0.1166499	best: 0.1166499 (3)	total: 152ms	remaining: 37.9s
4:	learn: 0.1450528	test: 0.1146680	best: 0.1146680 (4)	total: 169ms	remaining: 33.7s
5:	learn: 0.1424806	test: 0.1123631	best: 0.1123631 (5)	total: 205ms	remaining: 33.9s
6:	learn: 0.1400108	test: 0.1102799	best: 0.1102799 (6)	total: 240ms	remaining: 34s
7:	learn: 0.1372911	test: 0.1081941	best: 0.1081941 (7)	total: 278ms	remaining: 34.5s
8:	learn: 0.1350834	test: 0.1062066	best: 0.1062066 (8)	total: 313ms	remaining: 34.5s
9:	learn: 0.1327760	test: 0.1042844	best: 0.1042844 (9)	total: 348ms	remaining: 34.5s
10:	learn: 0.1304312	test: 0.1022763	best: 0.1022763 (10)	total: 384ms	remaining: 34.5s
11:	learn: 0.1280555	test: 0.1004625	best: 0.1004625 (11

In [0]:
test['is_holiday'] = test['application_date'].dt.date.isin(holidays)

In [0]:
add_datepart(test, 'application_date')

In [0]:
test['application_Is_month_end'] = test['application_Is_month_end'].astype(int)
test['application_Is_month_start'] = test['application_Is_month_start'].astype(int)
test['application_Is_quarter_end'] = test['application_Is_quarter_end'].astype(int)
test['application_Is_quarter_start'] = test['application_Is_quarter_start'].astype(int)
test['application_Is_year_end'] = test['application_Is_year_end'].astype(int)
test['application_Is_year_start'] = test['application_Is_year_start'].astype(int)

In [0]:
preds = model.predict(test.drop(['id'], axis=1))
preds[0:5]

array([7.96893779, 7.56978232, 8.23431505, 8.16041093, 8.18314118])

In [0]:
preds = np.expm1(preds)

In [0]:
test_sub = pd.read_csv('/content/drive/My Drive/AV LTFS Finhack2/test.csv', parse_dates=['application_date'])

In [0]:
sub = pd.DataFrame()
sub['id'] = test_sub['id']
sub['application_date'] = test_sub['application_date']
sub['segment'] = test_sub['segment']
sub['case_count'] = preds
sub.head()

,id,application_date,segment,case_count
0,1,2019-07-06,1,2888.786161
1,2,2019-07-07,1,1937.718206
2,3,2019-07-08,1,3767.058096
3,4,2019-07-09,1,3498.624425
4,5,2019-07-10,1,3579.082716


In [0]:
sub.to_csv('catboost_depth12.csv', index=False, header=True)